In [ ]:
from gurobipy import *

In [ ]:
m1 = Model('VRPTW')

In [ ]:
from operator import le
from unicodedata import name
from numpy import conjugate
import random
from gurobipy import *
m1 = Model('vrptw')

# 20 customer -> 1.1 minute
# 30 customers -> 5 minute    1.69 minutes
# 40                          2.45 min
# 50 customer -> 160 minute


def discreteTime(timeWindow, intervalSize):   # O(lenghtOftimeWindow/size)
    tw = []
    currtime = timeWindow[0]
    while (currtime <= timeWindow[1]):
        tw.append(currtime)
        currtime = currtime+intervalSize
    return tw

customers = 30

# timeWindows = [[20, 30], [30, 40], [30, 40]]

timeWindows = [[170, 200], [80, 210], [410, 490], [80, 190], [290, 350], [280, 390], [160, 260], [460, 530], [450, 640], [180, 190], [500, 550], [140, 340], [160, 290], [180, 240], [160, 310], [490, 580], [460, 560], [190, 350], [310, 500], [190, 320], [10, 140], [140, 320], [140, 250], [270, 470], [40, 170], [330, 520], [430, 490], [480, 610], [410, 600], [190, 300]]
timeMatrix = [[0, 30, 100, 60, 50, 20, 80, 70, 90, 60, 100, 40, 100, 50, 50, 50, 30, 10, 70, 80, 30, 30, 70, 60, 10, 60, 50, 20, 40, 20], [30, 0, 30, 40, 30, 40, 60, 90, 100, 10, 40, 50, 80, 70, 40, 20, 30, 70, 50, 40, 30, 70, 80, 20, 30, 90, 80, 30, 80, 60], [100, 30, 0, 30, 80, 100, 60, 10, 90, 20, 50, 60, 40, 100, 50, 70, 60, 60, 20, 70, 100, 80, 40, 50, 20, 80, 70, 50, 20, 70], [60, 40, 30, 0, 20, 30, 80, 20, 70, 40, 70, 60, 40, 70, 20, 100, 50, 70, 70, 80, 20, 70, 20, 10, 60, 60, 60, 60, 10, 90], [50, 30, 80, 20, 0, 20, 20, 20, 100, 60, 10, 40, 70, 100, 10, 50, 70, 70, 90, 80, 80, 60, 20, 100, 80, 20, 50, 80, 10, 80], [20, 40, 100, 30, 20, 0, 80, 20, 30, 30, 80, 10, 50, 10, 100, 20, 20, 20, 100, 80, 30, 30, 90, 40, 80, 10, 40, 30, 60, 40], [80, 60, 60, 80, 20, 80, 0, 60, 10, 10, 80, 50, 70, 80, 20, 50, 40, 20, 90, 90, 30, 70, 50, 50, 10, 20, 20, 60, 80, 30], [70, 90, 10, 20, 20, 20, 60, 0, 30, 20, 90, 70, 70, 20, 30, 50, 40, 80, 30, 50, 100, 50, 40, 90, 30, 70, 10, 50, 50, 50], [90, 100, 90, 70, 100, 30, 10, 30, 0, 100, 20, 40, 10, 30, 70, 80, 50, 80, 30, 20, 100, 80, 100, 30, 80, 30, 100, 10, 50, 40], [60, 10, 20, 40, 60, 30, 10, 20, 100, 0, 70, 20, 30, 70, 90, 20, 100, 70, 70, 80, 50, 90, 30, 40, 40, 40, 40, 60, 100, 10], [100, 40, 50, 70, 10, 80, 80, 90, 20, 70, 0, 20, 100, 100, 30, 30, 50, 60, 90, 20, 40, 100, 10, 50, 90, 30, 80, 70, 100, 30], [40, 50, 60, 60, 40, 10, 50, 70, 40, 20, 20, 0, 20, 60, 100, 60, 70, 80, 90, 50, 100, 40, 30, 60, 60, 90, 40, 30, 50, 60], [100, 80, 40, 40, 70, 50, 70, 70, 10, 30, 100, 20, 0, 40, 40, 80, 50, 90, 90, 100, 60, 50, 40, 20, 10, 60, 40, 20, 40, 50], [50, 70, 100, 70, 100, 10, 80, 20, 30, 70, 100, 60, 40, 0, 100, 30, 70, 20, 80, 70, 100, 60, 40, 10, 80, 20, 100, 10, 100, 100], [50, 40, 50, 20, 10, 100, 20, 30, 70, 90, 30, 100, 40, 100, 0, 100, 30, 20, 60, 50, 80, 90, 20, 60, 90, 80, 90, 90, 80, 10], [50, 20, 70, 100, 
50, 20, 50, 50, 80, 20, 30, 60, 80, 30, 100, 0, 40, 30, 40, 10, 80, 80, 10, 50, 50, 50, 90, 70, 30, 30], [30, 30, 60, 50, 70, 20, 40, 40, 50, 100, 50, 70, 50, 70, 30, 40, 0, 70, 50, 10, 10, 80, 50, 60, 100, 30, 30, 40, 100, 10], [10, 70, 60, 70, 70, 20, 20, 80, 80, 70, 60, 80, 90, 20, 20, 30, 70, 0, 40, 30, 80, 60, 100, 30, 40, 30, 90, 40, 30, 20], [70, 50, 20, 70, 90, 100, 90, 30, 30, 70, 90, 90, 90, 80, 60, 40, 50, 40, 0, 50, 40, 80, 50, 90, 30, 100, 100, 30, 100, 60], [80, 40, 70, 80, 80, 80, 90, 50, 20, 80, 20, 50, 100, 70, 50, 10, 10, 30, 50, 0, 60, 70, 20, 40, 80, 40, 90, 80, 90, 40], [30, 30, 100, 20, 80, 30, 30, 100, 100, 50, 40, 100, 60, 100, 80, 80, 10, 80, 40, 60, 0, 40, 30, 30, 20, 100, 100, 60, 80, 10], [30, 70, 80, 70, 60, 30, 70, 50, 80, 90, 100, 40, 50, 60, 90, 80, 80, 60, 80, 70, 40, 0, 10, 100, 20, 50, 70, 50, 30, 100], [70, 80, 40, 20, 20, 90, 50, 40, 100, 30, 10, 30, 40, 40, 20, 10, 50, 100, 50, 20, 30, 10, 0, 90, 50, 20, 40, 30, 100, 10], [60, 20, 50, 10, 100, 40, 50, 90, 30, 40, 50, 60, 20, 10, 
60, 50, 60, 30, 90, 40, 30, 100, 90, 0, 40, 80, 90, 50, 60, 50], [10, 30, 20, 60, 80, 80, 10, 30, 80, 40, 90, 60, 10, 80, 90, 50, 100, 40, 30, 80, 20, 20, 50, 40, 0, 70, 50, 10, 70, 50], [60, 90, 80, 60, 20, 10, 20, 70, 30, 40, 30, 90, 60, 20, 80, 50, 30, 30, 100, 40, 100, 50, 20, 80, 70, 0, 40, 40, 40, 20], [50, 80, 70, 60, 50, 40, 20, 10, 100, 40, 80, 40, 40, 100, 90, 90, 30, 90, 100, 90, 100, 70, 40, 90, 50, 40, 0, 80, 30, 10], [20, 30, 50, 60, 80, 30, 60, 50, 10, 60, 70, 30, 20, 10, 90, 70, 40, 40, 30, 80, 
60, 50, 30, 50, 10, 40, 80, 0, 50, 50], [40, 80, 20, 10, 10, 60, 80, 50, 50, 100, 100, 50, 40, 100, 80, 30, 100, 30, 100, 90, 80, 30, 100, 60, 70, 40, 30, 50, 0, 40], [20, 
60, 70, 90, 80, 40, 30, 50, 40, 10, 30, 60, 50, 100, 10, 30, 10, 20, 60, 40, 10, 100, 10, 50, 50, 20, 10, 50, 40, 0]]

# timeWindows = []
# for i in range(0,customers):
#     st = random.randint(1,2)*10
#     et = random.randint(1,5)*10 + st
#     tw = [st,et]
#     timeWindows.append(tw)

# print("time Windows ", timeWindows)
# print()

discreteTimeWindows = []
for i in timeWindows:
    discreteTimeWindows.append(discreteTime(i, 10))

print("discreteTimeWindows", discreteTimeWindows)
print()

# starting_time = '6:00am'

# timeMatrix = [[0, 30, 10], [30, 0, 10], [10, 10, 0]]

# timeMatrix = []
# for i in range(0,customers):
#     timeMatrix.append([0]*customers)
# for i in range(0,customers):
#     for j in range(0,customers):
#         if(i==j):
#             timeMatrix[i][j] = 0
#         elif (i<j):
#             timeMatrix[i][j] = random.randint(1,10)*10
#             timeMatrix[j][i] = timeMatrix[i][j]

# print("timeMatrix", timeMatrix)
# print()


In [ ]:


count = 0
nodeCount = []  # assigns a number to node using count and stores its value

for i in range(0,len(discreteTimeWindows)):
    c = []
    for j in range(0, len(discreteTimeWindows[i])):
        c.append(count+1)
        count += 1
    nodeCount.append(c)

print("Node Count ", nodeCount)
print()


edgeFlows = {}
edgeNumber = 0
x = [] 
edges = {}
edgesList = []  # stores edge number going out of ith node
edgesList.append([])  # for s
# solver = pywraplp.Solver.CreateSolver('GLOP')

In [ ]:
# Graph Construction

# out of s -> assuming every node is reachable from s
e =[]
for i in range(0,len(nodeCount)):
    for j in range(0,len(nodeCount[i])):
        e.append(edgeNumber)
        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber ))
        edges['x[%i]' % edgeNumber] = (0,nodeCount[i][j])
        edgeNumber+=1
edgesList[0]=e

for i in range(0,len(discreteTimeWindows)):
        for j in range(0,len(discreteTimeWindows[i])):    # j th node
            u = nodeCount[i][j]
            utime = discreteTimeWindows[i][j]
            e = []  # stores edges reachable from u
            for k in range(0,len(discreteTimeWindows)):
                for l in range(len(discreteTimeWindows[k])-1,-1,-1): # l th node
                    v = nodeCount[k][l]
                    vtime = discreteTimeWindows[k][l]
                    if(i!=k and utime+timeMatrix[i][k]<=vtime):
                        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber ))
                        edges['x[%i]' % edgeNumber] = (u,v) # u -> v reachable
                        e.append(edgeNumber)
                        edgeNumber += 1
                    elif (utime+timeMatrix[i][k]>vtime):
                        break
            edgesList.append(e)

# into t
e=[]
for i in range(0,len(nodeCount)):
    for j in range(0,len(nodeCount[i])):
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber ))
            edges['x[%i]' % edgeNumber] = (nodeCount[i][j],-1)
            #add edge number in edge list 
            edgesList[nodeCount[i][j]].append(edgeNumber)
            edgeNumber+=1
edgesList.append([])
print("Node Count after s and t:",nodeCount)
print()


m1.update()

print('edge list', edgesList)
print()
print("edges", edges)
print()
print('edge number', edgeNumber)
print()

In [ ]:
# constraints
noc = 0
# contraint for outgoing flow for a customer
   
for i in range(0,len(nodeCount)): # choose one list from nodeCount
    coeff = [] # store where coeff = 1
    for j in range(0,len(nodeCount[i])): # se
        node = nodeCount[i][j]
        for k in range(0,len(edgesList[node])):
            e = edgesList[node][k]
            coeff.append(x[e])
    m1.addConstr(( sum(coeff) == 1  ))
    noc+=1
        

# constraint for incoming for a customer
incomingEdgeList =[]
for i in range(0,len(edgesList)):
        incomingEdgeList.append([]) # adding lists in a list
for i in edges:
        v = edges[i][1]  # u -> v
        edge_no = int(i[2:-1])
        incomingEdgeList[v].append(edge_no)

print("incoming edge list: ",incomingEdgeList)
print()

for i in range(0,len(nodeCount)):
    contraint = [0] * (edgeNumber) # indicates number of edges
    coeff = []
    for j in range(0,len(nodeCount[i])):
        node = nodeCount[i][j]
        for k in range(0,len(incomingEdgeList[node])):
            e = incomingEdgeList[node][k]
            contraint[e] = 1
            coeff.append(x[e])

    m1.addConstr(( sum(coeff) == 1  ))
    noc+=1

# constraint for a node incoming = outgoing
for i in range(0,len(nodeCount)):
    for j in range(0,len(nodeCount[i])):
        contraint = [0] * (edgeNumber)
        coeff = []
        node = nodeCount[i][j]
        for k in range(0,len(incomingEdgeList[node])):
            e = incomingEdgeList[node][k]
            contraint[e] = 1
            coeff.append(x[e])
        for k in range(0,len(edgesList[node])):
            e = edgesList[node][k]
            contraint[e] = -1
            coeff.append(-x[e])
        m1.addConstr(( sum(coeff) == 0  ))
        noc+=1

In [ ]:
objectivefun = []
for i in range(0, len(edgesList[0])):
    objectivefun.append(x[edgesList[0][i]])
obj = sum(objectivefun)
m1.setObjective(obj,GRB.MINIMIZE)
                
    
m1.optimize()
print("Runtime is :",m1.Runtime)
print("objective value : ",m1.objVal)

In [ ]:
for v in m1.getVars():
    print('%s: %g' % (v.varName,v.x))

In [ ]:
print("Number of Constraints :",noc)